# Verlet Integration and Constraints (math reference)

**Verlet integration** and **per-axis constraints** (clamp position, reproject old position so velocity respects the boundary) as in `.planning/DECISIONS.md` (Narrative Engine). For **game context**, axis semantics (Courage, Loyalty, Hunger), and how Kaiza reaches dialogue options, see **[dungeonbreak-narrative.ipynb](dungeonbreak-narrative.ipynb)**.

**Rule:** After constraining `p` to [min, max], set `old_p` so that the next step's implied velocity is along the boundary (slide) or reflected (bounce).

**Reference:** [Narrative state vector concept](../docs/narrative/narrative-state-vector-concept.png) — the axes we constrain (e.g. Courage, Loyalty) are the same dimensions entities move in.

In [ ]:
from IPython.display import Image, display
display(Image(filename='../docs/narrative/narrative-state-vector-concept.png', width=550))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def verlet_step(p, old_p, a, dt):
    """One Verlet step: new_p = p + (p - old_p) + a * dt^2."""
    new_p = p + (p - old_p) + a * dt * dt
    return new_p

def constrain_and_reproject(p, old_p, p_min, p_max, slide=True):
    """Clamp p to [p_min, p_max]; set old_p so implied velocity respects boundary.
    If slide=True: reproject so velocity is along boundary (slide).
    If slide=False: reproject for bounce (reflect velocity)."""
    p_new = np.clip(p, p_min, p_max)
    # Implied velocity (before constraint) is v = p - old_p.
    # After constraint we want: next_p = p_new + (p_new - old_p_new) + ...
    # So implied v_next = p_new - old_p_new. For slide: we want v_next tangent to boundary.
    # Simple approach: old_p_new = p_new - (p - old_p) keeps same velocity; but if we clipped,
    # we want the component that pushed out to be zero. So set old_p so that p_new - old_p_new
    # has no component in the direction we clipped. For 1D: if p was clamped to max, set
    # old_p_new = 2*p_new - p (so v_implied = p_new - old_p_new = p - p_new, reversed).
    # Standard slide: old_p_new = p + (p_new - p) = p_new for the constrained dim? No.
    # Slide: new velocity should be zero along the normal we hit. So in 1D, if we hit max,
    # we want next position not to go further: set old_p_new = 2*p_new - old_p (reflect).
    # That gives v_implied = p_new - (2*p_new - old_p) = old_p - p_new. For bounce that's correct.
    # For slide we want v_implied = 0 on that axis: set old_p_new = p_new so v = 0.
    if slide:
        old_p_new = np.where(p != p_new, p_new, old_p)  # zero velocity on constrained axis
    else:
        old_p_new = np.where(p != p_new, 2 * p_new - old_p, old_p)  # bounce
    return p_new, old_p_new

## 2D particle with constant force and box constraint

Particle starts inside [0,1] x [0,1]; gravity-like force in -y. Constrain to box; slide when hitting walls.

In [ ]:
dt = 0.02
n_steps = 200
p_min = np.array([0., 0.])
p_max = np.array([1., 1.])
a = np.array([0.0, -0.5])  # slight pull downward

p = np.array([0.5, 0.9])
old_p = np.array([0.5, 0.92])  # initial velocity ~ (0, -0.02/dt) upward

path = [p.copy()]
for _ in range(n_steps):
    new_p = verlet_step(p, old_p, a, dt)
    new_p, old_p = constrain_and_reproject(new_p, p, p_min, p_max, slide=True)
    p = new_p
    path.append(p.copy())

path = np.array(path)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(path[:, 0], path[:, 1], 'b-', alpha=0.7, label='Trajectory')
ax.scatter(path[0, 0], path[0, 1], color='green', s=80, label='Start', zorder=5)
ax.scatter(path[-1, 0], path[-1, 1], color='red', s=80, label='End', zorder=5)
rect = plt.Rectangle((0, 0), 1, 1, fill=False, edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.1, 1.1)
ax.set_aspect('equal')
ax.set_xlabel('x (e.g. Courage)')
ax.set_ylabel('y (e.g. Loyalty)')
ax.set_title('Verlet + box constraint (slide at boundaries)')
ax.legend()
plt.tight_layout()
plt.show()

## Single axis constraint: velocity before and after clamp

1D: particle would go past 1.0; we clamp and reproject old_p so the next step doesn't push further out.

In [ ]:
p_1d = np.array([0.9])
old_p_1d = np.array([0.85])  # velocity +0.05 per step
a_1d = np.array([0.2])  # acceleration to the right

positions = [p_1d[0]]
for _ in range(30):
    new_p = verlet_step(p_1d, old_p_1d, a_1d, dt)
    new_p, old_p_1d = constrain_and_reproject(new_p, p_1d, np.array([0.]), np.array([1.]), slide=True)
    p_1d = new_p
    positions.append(p_1d[0])

plt.figure(figsize=(10, 3))
plt.plot(positions, 'o-')
plt.axhline(y=1.0, color='red', linestyle='--', label='Constraint max')
plt.xlabel('Step')
plt.ylabel('Position')
plt.title('1D Verlet with clamp at 1.0 (slide): position stays in bounds')
plt.legend()
plt.tight_layout()
plt.show()